In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:
# sourced from github

#create artificial training data
import numpy as np

X = np.random.normal(size = (10000, 10, 30))
y = np.random.binomial(1, 0.3, size = 10000)

In [2]:
# In[ ]:


#create dask arrays from numpy array
from dask.array import from_array
X = from_array(X, chunks = {0:1000, 1:10, 2:30})
y = from_array(y, chunks = 1000)

In [3]:
# In[ ]:


#print dask array X
X

dask.array<array, shape=(10000, 10, 30), dtype=float64, chunksize=(1000, 10, 30), chunktype=numpy.ndarray>

In [4]:
# In[ ]:


#print dask array y
y

dask.array<array, shape=(10000,), dtype=int64, chunksize=(1000,), chunktype=numpy.ndarray>

In [5]:
# In[ ]:


#build and compile TF model
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, SpatialDropout1D, BatchNormalization, Dropout
from tensorflow.keras.metrics import AUC

def nn():
    model_input = Input(shape = (10, 30))
    model_dropout1 = SpatialDropout1D(0.2)(model_input)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

2021-11-27 02:17:46.592180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 02:17:46.592241: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
# In[ ]:


#wrap TF model with KerasClassifier and incremental learner
from scikeras.wrappers import KerasClassifier
from dask_ml.wrappers import Incremental

model = KerasClassifier(model = nn)
model = Incremental(model)

In [7]:
# In[ ]:
from distributed import Client

#assume that dask scheduler and workers are already online and scaled

# client = Client('tcp://dask-scheduler.kubeflow-user.svc.cluster.local:8786')
client = Client('tcp://10.42.0.90:8786')
client

<Client: 'tcp://10.42.0.90:8786' processes=6 threads=12, memory=11.18 GiB>

In [8]:
# In[ ]:


model.fit(X, y, classes = [0, 1])


# In[ ]:


client.close()

ValueError: Expected 2D array, got 1D array instead:
array=['array-b590ca107d6036231858d556912b5d5e' '3' '0'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.